In [1]:
#对于2018年CIC2rd的反馈数据进行文本分类

In [8]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
from random import shuffle
import datetime

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

#bucket = sess.default_bucket() # Replace with your own bucket name if needed
bucket = 'ray-ai-ml' # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised/ticket' #Replace with the prefix under which you want to store the data if needed

arn:aws-cn:iam::876820548815:role/service-role/AmazonSageMaker-ExecutionRole-20200520T151303
ray-ai-ml


In [107]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple jieba

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [108]:
import jieba
import pandas as pd

In [6]:
index_to_label = {} 
with open("classes.txt") as f:
    for i,label in enumerate(f.readlines()):
        ll = label.strip().split(',')
        index_to_label[ll[0]] = ll[1]
print(index_to_label)

{'1001': '云端预开通', '1002': 'Carplay', '1003': 'change', '1004': '云端APP使用', '1005': 'RS', '1006': 'store', '1007': '云端APP显示', '1008': '购买问题', '1009': 'I-CALL', '10010': '远程3D视图', '10011': 'VIN无效', '10012': '收不到验证码', '10013': 'RTTI', '10014': 'wifi', '10015': '云端APP显示RS', '10016': 'CD', '10017': 'BON', '10019': 'APP登陆问题', '10020': 'GPS', '10021': 'Remote无内容', '10022': 'POI'}


In [3]:
import logging,os,jieba,re
#!wget https://cdc-code.s3.cn-north-1.amazonaws.com.cn/chineseStopWords.txt
def get_stopwords(StopWordFileName):
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)  
      #加载停用词表 
    stopword_set = set()
    with open(StopWordFileName,'r',encoding="utf-8") as stopwords:
        for stopword in stopwords: 
            stopword_set.add(stopword.strip("\n"))  
    return stopword_set

# Debug code
# stopwords = get_stopwords()
# i = 0
# for x in stopwords:
#     if i > 10:
#         break
#     print(x)
#     i += 1
    
def clear_timestamp(mystr):
    patterns = [r"\w{3} \w{3} \d{2} \d{1,2}:\d{1,2}:\d{1,2} \d{4}\s*",    #sun aug 19 13:02:10 2018
        r"\w{3}, \d{2} \w{3} \d{4} \d{1,2}:\d{1,2}:\d{1,2} \w{2}\s*",     #Sun, 19 Aug 2018 13:02:08 ET
        r"\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2}:\s*",                       #2018-11-01 09:35:
        r"\d{4}/\d{1,2}/\d{1,2}\s*",                                    #2018/9/1
        r"\d{1,2}/\d{1,2}/\d{4}\s*",                                    #9/1/2018
        r"\d{4}.\d{1,2}.\d{0,2}\s*",                                    #2018.9.1
        r"\d{1,2}.\d{1,2}.\d{4}\s*",                                    #9.1.2018
        r"\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2}:\d{4}/\d{1,2}/\d{1,2}\s*",      #2018-11-01 11:18:2018/10/31
        r"\d{1,2}:\d{1,2}:\d{4}/\d{1,2}/\d{1,2}\s*",      #2018-11-01 11:18:2018/10/31 21:09:08
        r"\d{1,2}:\d{1,2}:\d{1,2}\s*(AM|PM|am|pm)\s*",        #4:00:58 PM
        r"\d{1,2}:\d{1,2}:\d{1,2}\s*",                                     #21:09:08
        r"\d{1,2}:\d{1,2}\s*",                                     #21:09:08
        r"(\d{4})年(\d{1,2})月(\d{1,2})日\s*",                 #2018年10月5日
        r"(\d{2,4})年\s*",                 #2018年
        r"(\d{4})年(\d{1,2})月\s*",                 #2018年10月
        r":\s*([\da-zA_Z]+\/)+([a-zA-Z0-9\.]+)"                     #URL
        ]

    s = mystr

    for p in patterns:
        s = re.sub(p,'', s)

    s = s.strip()
    return s

# Debug code
# file  = 'wwww.txt'#'CIC2ND.txt'
# with open(file) as f:
#     lines = f.readlines()
    
# labels = []
# for line in lines:
#     s = clear_timestamp(line)
#     print(s)

def clear_email_phone_colon(mystr):
    patterns = [r"1[0-9]{10}", #mobile
        r"(\(0\d{2}\) \d{8})|(\(0\d{3}\) \d{7})|(\(0\d{3}\)-\d{8}$)|(\(0\d{2}\)\d{8})|(\(0\d{3}\)\d{7})|(\(0\d{3}\)\d{8}$)",                       #phone
        r"(0\d{2}-\d{8})|(0\d{3}-\d{7})|(0\d{3}-\d{8}$)|(\d{8})",                       #phone
        r"\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*"    #email
        ]

    s = mystr

    for p in patterns:
        s = re.sub(p,'', s)

    s = s.strip()
    return s

# Debug code
file  = 'wwww.txt'#'CIC2ND.txt'
with open(file) as f:
    lines = f.readlines()
    
labels = []
for line in lines:
    s = clear_email_phone_colon(line)
    print(s)

1005_!_RS_!_客户罗先生（车主黄先生的家人）致电，表示其i豪华型车辆，之前已经接到互联驾驶通知协议手机号更改成功的通知，但目前使用新协议手机号登陆云端互联APP后，首页不显示远程控制选项了（更改协议手机号之前可以正常使用），针对此问题，烦请互联驾驶部门跟进处理。 签订互联驾驶协议时登记的车主姓名协议手机号码：（原协议手机号） VIN：登陆云端互联密码：手机型号&版本：IOS 最新 云端互联APP版本：最新 联系人： 罗先生 联系电话：
1002_!_Carplay_!_ya_gao 2018-11-01 09:35: 客户杨先生通过在线客户平台反馈，表示在2018/9/1在淮安润宝行店内购买的 530Li 尊享型 豪华套装车辆，客户表示互联驾驶已经开通但是还是没有Apple Carplay的选项，已建议客户发送邮件，针对客户问题烦请互联驾驶人员跟进处理。 联系人/协议手机号码： 车主/联系人： 车架号：
1001_!_云端预开通_!_ya_gao 2018-11-01 11:18:2018/10/31 21:09:08客户先生致电表示2018年10月在上海绿地宝仕购买的430i，6位确认码已经给到购车经销商，实名制审核已经通过，但最后一步还是显示未激活，，烦请相关部门跟进处理。客户先生再次致电表示互联驾驶一直没有激活，希望可以尽快联系，就此事宜烦请互联驾驶跟进处理。针对此事宜，客户先生再次致电表示在2018年10月22日10:44分收到互联驾驶相关部门回复，但是没有及时接听，客户表示互联驾驶还是没有激活，烦请互联驾驶部门跟进处理。客户先生再次致电表示互联驾驶与其联系但是还是没有开通互联驾 联系电话：
10010_!_远程3D视图_!_ya_gao 2018-11-01 16:48: 11/1/2018 4:00:58 PM 客户先生致电表示2018.7.14购买的530Li车辆，远程3D视图无法正常使用，已建议其更新宝马服务尝试，客户尝试之后还是无法正常使用，已建议客户发送截图至互联驾驶邮箱，针对此问题烦请跟进处理。 客户先生致电表示未接到互联驾驶电话，客户对此不认可，就此事宜烦请互联驾驶跟进。 签订互联驾驶协议时登记的车主姓名：刘羡 协议手机号码  联系电话： 1
1008_!_购买问题_!_ya_gao 2018-11-01 12:55:11/1/20

### 使用分词

In [116]:
import jieba.posseg as pseg

file  = 'wwww.txt'#'CIC2ND.txt'
with open(file) as f:
    lines = f.readlines()

#get stopwords
stopwords = get_stopwords('chinesestopwords_test.txt')

begin = datetime.datetime.now()

#单机并行分词
jieba.enable_parallel(8)
#启动paddle模式
jieba.enable_paddle()
    
labels = []
counts = {}                 #计数{word，frequency}
for line in lines:
    label = []
    line = line.split('_!_')
    if line[0].strip('"').isdigit() and index_to_label[line[0].strip('"')]:
        label_code = index_to_label[line[0].strip('"')]
        label.append('__label__' + label_code)
        no_phone = clear_email_phone_colon(line[2])
        no_timestamp = clear_timestamp(no_phone)
        fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
        #print(fenci)
        #words = jieba.cut(fenci,cut_all=False)
        words = pseg.cut(fenci,use_paddle=True) #paddle模式
        #words = pseg.cut(fenci,use_paddle=True,cut_all=True)
        filled_words = set()
        for word, flag in words:
            #print('raw %s %s' % (word, flag))
            # 人名, 地名，机构，方位名词, 量词, 代词, 时间, 副词
            if (flag == 'nr' or flag == 'ns' or flag == 'PER' 
                or flag == 'LOC' or flag == 'ORG' or flag == 'f' or flag == 'r' or flag == 'q'
                or flag == 't' or flag == 'TIME' or flag == 'd'):
                #print('%s %s' % (word, flag))
                continue
            else:
                if word not in stopwords:          #不在停用词表中
                    if len(word) == 1:
                        continue
                    else:
                        counts[word] = counts.get(word, 0) + 1
                    filled_words.add(word)
        label.extend(filled_words)
        #print(label)
        labels.append(label)

shuffle(labels)

end = datetime.datetime.now()
print('data processing data %d' %(end - begin).seconds)
print(labels[0:5])

items = list(counts.items())
items.sort(key=lambda x: x[1], reverse=True)
for i in range(30):
    word, count = items[i]
    print("{:<10}{:>7}".format(word, count))

Paddle enabled successfully......
2020-07-16 11:01:13,622:DEBUG:Paddle enabled successfully......


data processing data 0
[['__label__远程3D视图', '发送', '建议', '远程', '530', '登记', '邮箱', '服务', '视图', '签订', '购买', '号码', '宝马', '手机', '致电', '电话', '驾驶', '截图', '跟进', '事宜', '车辆', '更新', '尝试', '姓名', '未接到', '互联', '认可'], ['__label__RS', '型号', '通知', '联系人', '登记', '云端', '接到', '显示', '驾驶部门', '登陆', '签订', '选项', '号码', '手机', '致电', '远程控制', '电话', '驾驶', '豪华型', '版本', '跟进', '密码', '更改', '手机号', '最新', '车辆', '首页', '姓名', '互联', '成功'], ['__label__云端APP使用', '补发', '发送', '咨询', '加急', '服务', '希望', '多辆', '催促', '显示', '邮件', '还没有', '批次', '人员', '中心', '致电', '驾驶', '检测', '解决', '跟进', '想要', '工作', '连接', '渠道', '互联', '功能', '转接'], ['__label__RTTI', '激动', '建议', '查询', '不到', '保存', '界面', '登记', '315', '服务', '屏蔽', '通讯录', '提供', '驾驶部门', '地图', '签订', '续费', '用车', '人员', '路况', '购买', '图标', '号码', '宝马', '手机', '致电', '驾驶', '网络', '跟进', '告知', '工作', '事宜', '实时', '车辆', '更新', '姓名', '情绪', '功能', '互联', '回复', '力量'], ['__label__Carplay', '发送', '反馈', '建议', '530', '联系人', '套装', '在线', '邮件', '选项', '尊享型', '人员', '购买', '号码', '手机', '驾驶', '跟进', '豪华', '车辆', '开通', '平台', '互联', '车架']]


In [11]:
# pip安装
# !pip install pyltp -i https://pypi.tuna.tsinghua.edu.cn/simple/
# 或者编译安装
# !git clone https://github.com/HIT-SCIR/pyltp
# !cd pyltp
# !git submodule init 
# !git submodule update
# !python setup.py install
# !easy_install pyltp/dist/pyltp-0.2.3-py3.6-linux-x86_64.egg

# 模型下载
# https://pyltp.readthedocs.io/zh_CN/latest/api.html#id10

# 利用哈工大LTP分配

In [10]:
from pyltp import Postagger
from pyltp import Segmentor
import os
import datetime

#get stopwords
stopwords = get_stopwords('chinesestopwords_test.txt')

begin = datetime.datetime.now()

# 分词
LTP_DATA_DIR = '/home/ec2-user/SageMaker/classification/bmwfeedback/ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

# 词性标注 https://ltp.readthedocs.io/zh_CN/v3.4.0/appendix.html
postagger = Postagger()
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
postagger.load(pos_model_path)
    
labels = []
counts = {}                 #计数{word，frequency}
for line in lines:
    label = []
    line = line.split('_!_')
    if line[0].strip('"').isdigit() and index_to_label[line[0].strip('"')]:
        label_code = index_to_label[line[0].strip('"')]
        label.append('__label__' + label_code)
        no_phone = clear_email_phone_colon(line[2])
        no_timestamp = clear_timestamp(no_phone)
        fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
        #print(fenci)
        words = segmentor.segment(fenci)
        postags = postagger.postag(words)
        filled_words = set()
        for i in range(len(words)):
            word = words[i]
            flag = postags[i]
            #print('raw %s %s' % (words[i], postags[i]))
            # 人名nh, 地名nl, ns，机构ni，方位名词nd, 量词m, 代词r, 时间nt, 副词u,z
            if (flag == 'nh' or flag == 'ns' or flag == 'nl' 
                or flag == 'ni' or flag == 'nd' or flag == 'm' or flag == 'r' or flag == 'nt'
                or flag == 'u' or flag == 'z'):
                #print('%s %s' % (word, flag))
                continue
            else:
                if word not in stopwords:          #不在停用词表中
                    if len(word) == 1:
                        continue
                    else:
                        counts[word] = counts.get(word, 0) + 1
                    filled_words.add(word)
        label.extend(filled_words)
        #print(label)
        labels.append(label)

shuffle(labels)

end = datetime.datetime.now()
print('data processing data %d' %(end - begin).seconds)
print(labels[0:5])

items = list(counts.items())
items.sort(key=lambda x: x[1], reverse=True)
for i in range(30):
    word, count = items[i]
    print("{:<10}{:>7}".format(word, count))

segmentor.release() # 释放模型
postagger.release() # 释放模型

data processing data 0
[['__label__云端APP使用', '解决', '跟进', '加急', '发送', '博瑞宝', '中心', '邮件', '检测', '咨询', '渠道', '人员', '互联', '催促', '驾驶', '服务', '致电', '再次', '希望', '补发', '连接', '工作', '显示', '功能'], ['__label__RTTI', '事宜', '界面', '实时', '车辆', '激动', '跟进', '签订', '号码', '回复', '宝马', '网络', '用车', '力量', '分钟', '情绪', '保存', '人员', '互联', '屏蔽', '驾驶', '提供', '部门', '服务', '致电', '再次', '登记', '地图', '更新', '路况', '通讯录', '告知', '建议', '查询', '图标', '购买', '工作', '续费', '手机', '姓名', '功能'], ['__label__购买问题', '基础', '事宜', '资讯', '车辆', '跟进', '号码', '回复', '宝马', '方式', '咨询', '互联', '在线', '官网', '驾驶', '服务', '致电', '延长', '购买', '远程', '续费', '联系人', '手机', '小时', '认可', '云端'], ['__label__RS', '密码', '接到', '控制', '手机号', '电话', '车辆', '跟进', '签订', '号码', '型号', '最新', '登陆', '互联', '更改', '驾驶', '部门', '致电', '登记', '版本', '选项', '远程', '通知', '显示', '联系人', '手机', '成功', '姓名', '云端', '豪华型'], ['__label__Carplay', '平台', '润宝', '豪华', '反馈', '车辆', '跟进', '号码', '发送', '邮件', '人员', '互联', '在线', '车架', '驾驶', '享型', '建议', '购买', '联系人', '手机', '套装', '开通']]
互联             29
驾驶             24
致电    

In [191]:
prefix = 'blazingtext/ticket'

In [52]:
t_train_data = labels[0:int(len(labels)*0.8)]
t_validation_data = labels[int(len(labels)*0.8):]

In [160]:
#t_train_data[0:13]

In [54]:
import csv
t_train_file = 'tt.train'
t_validation_file = 'tt.validation'

with open(t_train_file, 'w') as csvoutfile:
    csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
    csv_writer.writerows(t_train_data)
    
with open(t_validation_file, 'w') as csvoutfile:
    csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
    csv_writer.writerows(t_validation_data)

In [55]:
%%time

t_train_channel = prefix + '/train'
t_validation_channel = prefix + '/validation'

sess.upload_data(path='tt.train', bucket=bucket, key_prefix=t_train_channel)
sess.upload_data(path='tt.validation', bucket=bucket, key_prefix=t_validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, t_train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, t_validation_channel)

s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

CPU times: user 244 ms, sys: 17.1 ms, total: 261 ms
Wall time: 563 ms


In [56]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [57]:
region_name = boto3.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

Using SageMaker BlazingText container: 390948362332.dkr.ecr.cn-north-1.amazonaws.com.cn/blazingtext:latest (cn-north-1)


In [58]:
t_bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 120,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)
t_bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

In [59]:
t_train_data = sagemaker.inputs.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
t_validation_data = sagemaker.inputs.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
t_data_channels = {'train': t_train_data, 'validation': t_validation_data}

In [60]:
t_bt_model.fit(inputs=t_data_channels, logs=True)

2020-07-03 06:15:10 Starting - Starting the training job...
2020-07-03 06:15:12 Starting - Launching requested ML instances......
2020-07-03 06:16:20 Starting - Preparing the instances for training......
2020-07-03 06:17:33 Downloading - Downloading input data...
2020-07-03 06:18:06 Training - Training image download completed. Training in progress..Arguments: train
[07/03/2020 06:18:06 WARNING 139882388739904] Loggers have already been setup.
[07/03/2020 06:18:06 WARNING 139882388739904] Loggers have already been setup.
[07/03/2020 06:18:06 INFO 139882388739904] nvidia-smi took: 0.0252678394318 secs to identify 0 gpus
[07/03/2020 06:18:06 INFO 139882388739904] Running single machine CPU BlazingText training using supervised mode.
[07/03/2020 06:18:06 INFO 139882388739904] Processing /opt/ml/input/data/train/tt.train . File size: 20 MB
[07/03/2020 06:18:06 INFO 139882388739904] Processing /opt/ml/input/data/validation/tt.validation . File size: 5 MB
Read 3M words
Number of words:  4432

In [61]:
t_text_classifier = t_bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.t3.medium')

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: 1 validation error detected: Value 'ml.t3.medium' at 'productionVariants.1.member.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.r5d.12xlarge, ml.r5.12xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.p3.16xlarge, ml.m5d.xlarge, ml.m5.large, ml.t2.xlarge, ml.p2.16xlarge, ml.m5d.12xlarge, ml.inf1.2xlarge, ml.m5d.24xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.inf1.6xlarge, ml.c5d.2xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c5d.4xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.c4.large, ml.c5d.xlarge, ml.c5.large, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5d.large, ml.c5.xlarge, ml.c5d.9xlarge, ml.c4.xlarge, ml.inf1.xlarge, ml.g4dn.8xlarge, ml.inf1.24xlarge, ml.m5d.2xlarge, ml.t2.2xlarge, ml.c5d.18xlarge, ml.m5d.4xlarge, ml.t2.medium, ml.c5.18xlarge, ml.r5d.2xlarge, ml.r5.2xlarge,

In [159]:
sentences = "邓超加入春晚"#"宝马推出新车型/亚马逊云计算Q1营收过百亿/美国航空母舰开往伊朗波斯湾/北京迎来黄金周小高峰/C罗纳尔多还是梅西"

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [' '.join(jieba.cut(sentences,cut_all=False))]

payload = {"instances" : tokenized_sentences}

# payload = {"instances" : tokenized_sentences,
#           "configuration": {"k": 2}}

t_response = t_text_classifier.predict(json.dumps(payload))

t_predictions = json.loads(t_response)
print(json.dumps(t_predictions, indent=2))
t_predictions[0]['label']

NameError: name 't_text_classifier' is not defined